In [99]:
import re
import nltk

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder

from tqdm.auto import tqdm

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Функция обучения

In [106]:
def train(model, optimizer, criterion, n_epochs, train_loader, test_loader):

  loss_train = []
  accuracy_train = []

  for epoch in range(n_epochs):
    model.train()
    for batch in tqdm(train_loader, desc=f"Training epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)

        loss = criterion(output, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval()

    correct = 0
    total = 0

    with torch.no_grad():
      for batch in tqdm(test_loader, desc=f"Testing epoch {epoch + 1}/{n_epochs}"):
        inputs = batch["input"]
        labels = batch["label"]
        output = model(inputs)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    accuracy_train.append(test_accuracy)

    print('Epoch [{}/{}], Loss: {:.4f}, Test Accuracy: {:.2f}%'.format(epoch + 1, n_epochs, loss.item(), test_accuracy * 100))
    loss_train.append(loss.item())


## 1. Представление и предобработка текстовых данных

### 1.1 Операции по предобработке:
* токенизация
* стемминг / лемматизация
* удаление стоп-слов
* удаление пунктуации
* приведение к нижнему регистру
* любые другие операции над текстом

In [3]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer

In [4]:
text = 'Select your preferences and run the install command. Stable represents the most currently tested and supported version of PyTorch. Note that LibTorch is only available for C++'

Реализовать функцию `preprocess_text(text: str)`, которая:
* приводит строку к нижнему регистру
* заменяет все символы, кроме a-z, A-Z и знаков .,!? на пробел


In [5]:
def preprocess_text(text: str):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z .,!?]',' ', text)
    return text


In [6]:
preprocess_text(text)

'select your preferences and run the install command. stable represents the most currently tested and supported version of pytorch. note that libtorch is only available for c  '

### 1.2 Представление текстовых данных при помощи бинарного кодирования


Представить первое предложение из `text` в виде тензора `sentence_t`: `sentence_t[i] == 1`, если __слово__ с индексом `i` присуствует в предложении.

In [7]:
def one_hot_sen(text: str) -> torch.Tensor():
    #vocab of unique words
    vocab = list(set(word_tokenize(text)))

    #First sentence
    sen1 = sent_tokenize(text)[0]

    #zero tensor
    tensor_sen = torch.zeros(len(vocab))

    for word in word_tokenize(sen1):
        tensor_sen[vocab.index(word)] = 1

    return tensor_sen


In [8]:
one_hot_sen(text)

tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
        1., 1., 1., 0., 0., 0., 0., 1.])

## 2. Классификация фамилий по национальности

Датасет: https://disk.yandex.ru/d/owHew8hzPc7X9Q?w=1

2.1 Считать файл `surnames/surnames.csv`.

2.2 Закодировать национальности числами, начиная с 0.

2.3 Разбить датасет на обучающую и тестовую выборку

2.4 Реализовать класс `Vocab` (токен = __символ__)

2.5 Реализовать класс `SurnamesDataset`

2.6. Обучить классификатор.

2.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: прогнать несколько фамилий студентов группы через модели и проверить результат. Для каждой фамилии выводить 3 наиболее вероятных предсказания.

### 2.1 Считать файл `surnames/surnames.csv`.


In [10]:
surnames_df = pd.read_csv('surnames.csv')

In [11]:
surnames_df.head()

,surname,nationality
0,Woodford,English
1,Coté,French
2,Kore,English
3,Koury,Arabic
4,Lebzak,Russian


### 2.2 Закодировать национальности числами, начиная с 0.


In [12]:
le = LabelEncoder()
le.fit(surnames_df['nationality'])
surnames_df['nationality'] = le.transform(surnames_df['nationality'])
surnames_df.head()

,surname,nationality
0,Woodford,4
1,Coté,5
2,Kore,4
3,Koury,0
4,Lebzak,14


In [14]:
print(le.classes_)

['Arabic' 'Chinese' 'Czech' 'Dutch' 'English' 'French' 'German' 'Greek'
 'Irish' 'Italian' 'Japanese' 'Korean' 'Polish' 'Portuguese' 'Russian'
 'Scottish' 'Spanish' 'Vietnamese']


### 2.3 Разбить датасет на обучающую и тестовую выборку


In [13]:
X_train, X_test, y_train, y_test = train_test_split(surnames_df['surname'], surnames_df['nationality'] , test_size=0.25, random_state=42)

### 2.4 Реализовать класс `Vocab` (токен = __символ__)


In [53]:
class Vocab:
    def __init__(self, data: pd.Series):
        self.data = data
        self.token_to_idx = self.create_vocab(self.data)
        self.idx_to_token = {token: symbol for symbol, token  in self.token_to_idx.items()}
        self.vocab_len = len(self.token_to_idx)


    def create_vocab(self, data):
        token_to_idx = {}
        str_data = " ".join(data)
        tokenizer = 0
        for symbol in str_data:
            if symbol not in token_to_idx:
                token_to_idx[symbol] = tokenizer
                tokenizer += 1

        return token_to_idx


In [16]:
vocab = Vocab(X_train)
vocab.token_to_idx

{'O': 0,
 's': 1,
 'u': 2,
 'l': 3,
 'i': 4,
 'v': 5,
 'a': 6,
 'n': 7,
 ' ': 8,
 'S': 9,
 'e': 10,
 'o': 11,
 'w': 12,
 'D': 13,
 'r': 14,
 't': 15,
 'E': 16,
 'j': 17,
 'd': 18,
 'I': 19,
 'A': 20,
 'h': 21,
 'M': 22,
 'f': 23,
 'G': 24,
 'k': 25,
 'L': 26,
 'y': 27,
 'H': 28,
 'm': 29,
 'J': 30,
 'R': 31,
 'g': 32,
 'P': 33,
 'b': 34,
 'T': 35,
 'c': 36,
 'K': 37,
 'B': 38,
 'W': 39,
 'z': 40,
 'p': 41,
 'q': 42,
 'Y': 43,
 'N': 44,
 'V': 45,
 'F': 46,
 'C': 47,
 'U': 48,
 'Z': 49,
 'ñ': 50,
 'ò': 51,
 'Q': 52,
 'x': 53,
 'X': 54,
 "'": 55,
 'é': 56,
 'ü': 57,
 'ó': 58,
 'á': 59,
 'ö': 60,
 'à': 61,
 'ã': 62,
 'ß': 63,
 'è': 64,
 ':': 65,
 'ä': 66,
 '1': 67,
 'í': 68,
 'ú': 69,
 '-': 70,
 'É': 71,
 'Ż': 72,
 'õ': 73,
 'ç': 74,
 'Ś': 75,
 'ì': 76,
 '/': 77,
 'ń': 78,
 'ê': 79,
 'Á': 80}

### 2.5 Реализовать класс `SurnamesDataset`

In [71]:
class SurnamesDataset(Dataset):
    def __init__(self, X, y, vocab: Vocab):
        self.X = X
        self.y = y
        self.vocab = vocab

    def vectorize(self, surname):
        '''Генерирует представление фамилии surname в при помощи бинарного кодирования (см. 1.2)'''
        surname_tensor = torch.zeros(self.vocab.vocab_len)

        for symbol in surname:
            surname_tensor[self.vocab.token_to_idx[symbol]] = 1

        return surname_tensor

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return {"input" : self.vectorize(self.X[idx]) , "label" : self.y[idx]}

In [72]:
vocab = Vocab(surnames_df['surname'])

surnames_dataset_train = SurnamesDataset(X_train.values, y_train.values, vocab)
surnames_dataset_test= SurnamesDataset(X_test.values, y_test.values, vocab)

### 2.6. Обучить классификатор.

In [73]:
surnames_dataloader_train = DataLoader(surnames_dataset_train, batch_size = 64, shuffle= True)
surnames_dataloader_test = DataLoader(surnames_dataset_test, shuffle= True)

In [129]:
class Classficator(nn.Module):
  def __init__(self, input_dim, num_classes):
        super(Classficator, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

        self.relu = nn.ReLU()

  def forward(self, x):
    x = self.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.relu(self.fc3(x))
    return x

In [130]:
surnames_dataloader_train.dataset[0]

{'input': tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
         1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 'label': 4}

In [131]:
num_classes = len(set(surnames_dataloader_train.dataset.y)) # 18
input_dim = len(surnames_dataloader_train.dataset[0]['input']) # 85
n_epochs = 5

model = Classficator(input_dim, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train(model, optimizer, criterion, n_epochs, surnames_dataloader_train, surnames_dataloader_test)

Training epoch 1/5:   0%|          | 0/129 [00:00<?, ?it/s]

Testing epoch 1/5:   0%|          | 0/2745 [00:00<?, ?it/s]

Epoch [1/5], Loss: 2.2885, Test Accuracy: 36.94%


Training epoch 2/5:   0%|          | 0/129 [00:00<?, ?it/s]

Testing epoch 2/5:   0%|          | 0/2745 [00:00<?, ?it/s]

...

Testing epoch 4/5:   0%|          | 0/2745 [00:00<?, ?it/s]

Epoch [4/5], Loss: 2.0160, Test Accuracy: 47.94%


Training epoch 5/5:   0%|          | 0/129 [00:00<?, ?it/s]

Testing epoch 5/5:   0%|          | 0/2745 [00:00<?, ?it/s]

Epoch [5/5], Loss: 1.5343, Test Accuracy: 48.56%


In [144]:
model.eval()
output = model(surnames_dataset_train.vectorize("Anpilov"))

output

label =


tensor([0.0000, 0.0000, 1.8247, 0.0000, 3.6074, 0.0000, 0.0000, 0.0000, 0.0000,
        3.4731, 0.0000, 0.0000, 0.0000, 0.0000, 5.7327, 0.0000, 0.2616, 0.0000],
       grad_fn=<ReluBackward0>)

## 3. Классификация обзоров ресторанов

Датасет: https://disk.yandex.ru/d/nY1o70JtAuYa8g

3.1 Считать файл `yelp/raw_train.csv`. Оставить от исходного датасета 10% строчек.

3.2 Воспользоваться функцией `preprocess_text` из 1.1 для обработки текста отзыва. Закодировать рейтинг числами, начиная с 0.

3.3 Разбить датасет на обучающую и тестовую выборку

3.4 Реализовать класс `Vocab` (токен = слово)

3.5 Реализовать класс `ReviewDataset`

3.6 Обучить классификатор

3.7 Измерить точность на тестовой выборке. Проверить работоспособность модели: придумать небольшой отзыв, прогнать его через модель и вывести номер предсказанного класса (сделать это для явно позитивного и явно негативного отзыва)


In [ ]:
class Vocab:
  def __init__(self, data):
    self.idx_to_token = ...
    self.token_to_idx = ...
    self.vocab_len = ...

In [ ]:
class ReviewDataset(Dataset):
  def __init__(self, X, y, vocab: Vocab):
    self.X = X
    self.y = y
    self.vocab = vocab

  def vectorize(self, review):
    '''Генерирует представление отзыва review при помощи бинарного кодирования (см. 1.2)'''

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return ...